<a href="https://colab.research.google.com/github/RoomRooms/2023BigData/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Test")